Things that could be added:

 - Comparisons between Roe and HLL
 - Comparisons showing full PyClaw runs with these solvers

# Approximate solvers for the shallow water equations

$\newcommand{\wave}{{\mathcal W}}$

In this chapter we discuss approximate Riemann solvers for a system of equations; namely, the shallow water equations:

\begin{align}
    h_t + (hu)_x & = 0 \label{SW_mass} \\
    (hu)_t + \left(hu^2 + \frac{1}{2}gh^2\right)_x & = 0. \label{SW_mom}
\end{align}

We investigated the exact solution of the Riemann problem for this system in [a previous chapter](shallow_water.ipynb).  We found that the system has two genuinely nonlinear characteristic fields, so the solution consists of two waves, each of which may be a shock or a rarefaction.

Recall that the flux Jacobian is

\begin{align}
f'(q) & = \begin{pmatrix} 0 & 1 \\ -(q_2/q_1)^2 + g q_1 & 2 q_2/q_1 \end{pmatrix} 
        = \begin{pmatrix} 0 & 1 \\ -u^2 + g h & 2 u \end{pmatrix}.
\end{align}

Its eigenvalues are

\begin{align} \label{SW:char-speeds}
    \lambda_1 & = u - \sqrt{gh} & \lambda_2 & = u + \sqrt{gh},
\end{align}

with corresponding eigenvectors

\begin{align} \label{SW:fjac-evecs}
    r_1 & = \begin{bmatrix} 1 \\ u-\sqrt{gh} \end{bmatrix} &
    r_2 & = \begin{bmatrix} 1 \\ u+\sqrt{gh} \end{bmatrix}
\end{align}


## Roe solver

To define a linearized Riemann solver, we consider the quasilinear form $q_t + f'(q)q_x$ and linearize by replacing the argument $q$ of the flux Jacobian with a fixed value $\hat{q}$.  For the shallow water system, this leads to

\begin{align} \label{SW_quasilinear}
h_t + (hu)_x & = 0 \\
(hu)_t + (g\hat{h}-\hat{u}^2)h_x + 2\hat{u}(hu)_x & = 0. \nonumber
\end{align}

Observe that \eqref{SW_quasilinear} is linear *not in the physical variables $h, u$, but in the conserved variables $h, hu$*.  Similarly, we have defined here $\hat{u} = \widehat{hu}/\hat{h}$.

The characteristic speeds of this linear hyperbolic system are

\begin{align} \label{SW:avg-char-speeds}
    s_1 & = \hat{u} - \sqrt{g\hat{h}} & s_2 & = \hat{u} + \sqrt{g\hat{h}},
\end{align}

with corresponding eigenvectors

\begin{align} \label{SW:avg-fjac-evecs}
    r_1(\hat{q}) & = \begin{bmatrix} 1 \\ \hat{u}-\sqrt{g\hat{h}} \end{bmatrix} &
    r_2(\hat{q}) & = \begin{bmatrix} 1 \\ \hat{u}+\sqrt{g\hat{h}} \end{bmatrix}
\end{align}

A linearized solver for the shallow water system simply decomposes the jump $q_r-q_l$ in terms
of the eigenvectors, to define the waves:

\begin{align}
q_r - q_l = \alpha_1 r_1 + \alpha_2 r_2 = \wave_1 + \wave_2
\end{align}

and uses the speeds given by \eqref{SW:avg-char-speeds}.  We refer to $\alpha_1, \alpha_2$ as the
wave strengths; they can be found by solving the system above, which amounts to

\begin{align}
\alpha = R(q)^{-1}(q_r-q_l),
\end{align}

where $R(q) = (r_1(q) | r_2(q)$ is the matrix of right eigenvectors, and thus $R(q)^{-1}$ is the matrix whose rows are the left eigenvectors.  This leads to the expressions

\begin{align}
\alpha_1 & = \frac{(\hat{u}+ \hat{c}) \delta_1 - \delta_2}{2\hat{c}} \\
\alpha_2 & = \frac{-(\hat{u} - \hat{c}) \delta_1 + \delta_2}{2\hat{c}},
\end{align}

where $\hat{c} = \sqrt{g\hat{h}}$ and $\delta = q_r -q_l$.

It can be checked directly that, regardless of how $\hat{h}, \hat{u}$ are chosen, we have the property

\begin{align}
s_1 \wave_1 + s_2 \wave_2 & = f(q_r) - f(q_l),
\end{align}

so this Riemann solver will lead to a conservative numerical scheme. *(is there a nice proof of this property in general when the linearization is based on $f'(\hat{q})$?)*

It remains only to specify the state $(\hat{h},\hat{u})$.
Evidently this state should be some kind of average of the states $q_l, q_r$.  As discussed in [the last chapter](burgers_approximate.ipynb), the Roe average can be defined by the property that it yields the exact solution to the Riemann problem when that solution consists of a single shock.  This property is quite useful, since at most points (in space and time) where a shock appears, it is an isolated shock corresponding to one characteristic family.  Only at isolated points in space and time do we have multiple shocks interacting.

For Burgers' equation, we found that the Roe average corresponded to a simple arithmetic average, but for other systems (like the one considered here) the Roe average is typically not the arithmetic average.  We refer the reader to (cite Roe's paper and Ch. 15 of FVMHP) for a derivation of the Roe average, which for this system is

\begin{align}
\hat{h} & = \frac{h_r + h_l}{2} \\
\hat{u} & = \frac{\sqrt{h_r} u_r + \sqrt{h_l} u_l}{\sqrt{h_r} + \sqrt{h_l}}.
\end{align}

In the cell below is an implementation of the basic Roe solver just described.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
figsize =(8,4)
mpl.rcParams['figure.figsize'] = figsize
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from exact_solvers import shallow_water
from collections import namedtuple
from utils import riemann_tools
from ipywidgets import interact
from ipywidgets import widgets, FloatSlider, Checkbox, RadioButtons, fixed
State = namedtuple('State', shallow_water.conserved_variables)
Primitive_State = namedtuple('PrimState', shallow_water.primitive_variables)

In [ ]:
def shallow_water_roe(q_l, q_r, grav=1.):
    """
    Approximate Roe solver for the shallow water equations.
    """
    
    h_l = q_l[0]
    hu_l = q_l[1]
    u_l = hu_l/h_l
    h_r = q_r[0]
    hu_r = q_r[1]
    u_r = hu_r/h_r
    
    delta = q_r - q_l
    
    # Roe averages
    h_hat = (h_r + h_l)/2.
    u_hat = (np.sqrt(h_r)*u_r + np.sqrt(h_l)*u_l) / (np.sqrt(h_r) + np.sqrt(h_l))
    c_hat = np.sqrt(grav*h_hat)
    
    s1 = u_hat - c_hat
    s2 = u_hat + c_hat
    
    alpha1 = ( (u_hat+c_hat)*delta[0] - delta[1])/(2*c_hat)
    alpha2 = (-(u_hat-c_hat)*delta[0] + delta[1])/(2*c_hat)
    
    h_m = q_l[0] + alpha1
    hu_m = q_l[1] + alpha1 * (u_hat - c_hat)
    q_m = np.array([h_m, hu_m])
    
    states = np.column_stack([q_l,q_m,q_r])
    speeds = [s1, s2]
    wave_types = ['contact','contact']
    
    def reval(xi):
        h_out  = (xi<s1)*h_l + (s1<=xi)*(xi<=s2)*h_m + (s2<xi)*h_r
        hu_out = (xi<s1)*hu_l + (s1<=xi)*(xi<=s2)*hu_m + (s2<xi)*hu_r
        return h_out, hu_out
    
    return states, speeds, reval, wave_types

Here is an example of the Roe solver, compared with the exact Riemann solution.

In [ ]:
# Dam-break example
h_l = 4; h_r = 1
u_l = 0; u_r = 0

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_roe(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

Notice that the rarefaction wave is approximated by a discontinuity..  Here is what the Roe solution looks like in the phase plane, with the true integral curve and Hugoniot locus also plotted for comparison.

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

The next example illustrates the exactness property for isolated shocks.  Here we have chosen $q_l$ and $q_r$ such that they are related by a single 2-shock (up to rounding errors).

In [ ]:
h_l = 2.20698770767; h_r = 1.
u_l = 2.27057814896/h_l; hu_r = 0.

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_roe_with_efix(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 #['Exact','Roe with fix','Roe w/o fix'],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In the first plot, the speed of the 1-wave appears different for the Roe solution and the exact solution, but this is unimportant since the 1-wave has zero strength.

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

### Entropy fix

As discussed previously, the use of an entropy-violating discontinuity in place of a rarefaction can be problematic in the case of a transonic rarefaction.  For the Roe solver, we can again apply an entropy fix, splitting the single discontinuity into two waves, to avoid this problem.  Several different entropy fixes have been proposed; here we focus on one that is quite general and effective in practice.

Suppose that $\lambda_1(q_l) < 0$ while $\lambda_1(q_m)>0$.  In this case, it appears that the 1-wave should be a transonic rarefaction.  We take the 1-wave $\wave^1$ of the Roe solver, with speed $s_1$, and split it into two waves, $\wave^1_l = \alpha^1_l r_1$ and $\wave^1_r = \alpha^1_r r_1$, with speeds $\lambda_1(q_l), \lambda_1(q_m)$.  Two requirements determine the strength of these waves.  First, the total jump across them should be equal to the jump across the original wave:

$$\wave^1_l + \wave^1_r = \wave^1.$$

Second, we must maintain conservation:

$$\lambda_1(q_l) \wave^1_l + \lambda_1(q_m) \wave^1_r = s^1 \wave^1.$$

This leads to

\begin{align*}
\wave^1_l & = \frac{\lambda^1(q_m)-s_1}{\lambda^1(q_m)-\lambda^1(q_l)}\wave^1 \\
\wave^1_r & = \frac{s_1 - \lambda^1(q_l)}{\lambda^1(q_m)-\lambda^1(q_l)}\wave^1.
\end{align*}

We can check in a similar way for a transonic 2-rarefaction, and modify the 2-wave accordingly.  Note that only one of the two characteristic fields can be transonic in the Riemann solution for the shallow water equations.

This entropy fix is implemented in the solver below.

In [ ]:
def shallow_water_roe_with_efix(q_l, q_r, grav=1., efix=True):
    """
    Approximate Roe solver for the shallow water equations,
    with entropy fix.
    """
    
    h_l = q_l[0]
    hu_l = q_l[1]
    u_l = hu_l/h_l
    h_r = q_r[0]
    hu_r = q_r[1]
    u_r = hu_r/h_r
    
    delta = q_r - q_l
    
    # Roe averages
    h_hat = (h_r + h_l)/2.
    u_hat = (np.sqrt(h_r)*u_r + np.sqrt(h_l)*u_l) / (np.sqrt(h_r) + np.sqrt(h_l))
    c_hat = np.sqrt(grav*h_hat)
    
    s1 = u_hat - c_hat
    s2 = u_hat + c_hat
    
    alpha1 = ( (u_hat+c_hat)*delta[0] - delta[1])/(2*c_hat)
    #alpha2 = (-(u_hat-c_hat)*delta[0] + delta[1])/(2*c_hat)
    
    h_m = q_l[0] + alpha1
    hu_m = q_l[1] + alpha1 * (u_hat - c_hat)
    u_m = hu_m/h_m
    q_m = np.array([h_m, hu_m])
        
    transonic = None
    if efix:
        # Check if either wave appears transonic
        lambda_1_l = u_l - np.sqrt(grav*h_l)
        lambda_1_m = u_m - np.sqrt(grav*h_m)
        lambda_2_m = u_m + np.sqrt(grav*h_m)
        lambda_2_r = u_r + np.sqrt(grav*h_r)
        
        if lambda_1_l < 0 < lambda_1_m: # 1-wave appears transonic
            transonic = 1
        elif lambda_2_m < 0 < lambda_2_r: # 2-wave appears transonic
            transonic = 2
    
    if transonic == 1:
        beta = (lambda_1_m - s1)/(lambda_1_m - lambda_1_l)
        alpha1_l = beta*alpha1
        #alpha_1_r = (1-beta)*alpha1
        h_m_new = q_l[0] + alpha1_l
        hu_m_new = q_l[1] + alpha1_l * (u_hat-c_hat)
        q_m_new = np.array([h_m_new, hu_m_new])
        states = np.column_stack([q_l,q_m_new,q_m,q_r])
        speeds = [lambda_1_l, lambda_1_m, s2]
        wave_types = ['contact']*3
        
        def reval(xi):
            h_out  = (xi<speeds[0])*states[0,0] + \
                     (speeds[0]<=xi)*(xi<speeds[1])*states[0,1] + \
                     (speeds[1]<=xi)*(xi<speeds[2])*states[0,2] + \
                     (speeds[2]<=xi)*states[0,3]
            hu_out = (xi<speeds[0])*states[1,0] + \
                     (speeds[0]<=xi)*(xi<speeds[1])*states[1,1] + \
                     (speeds[1]<=xi)*(xi<speeds[2])*states[1,2] + \
                     (speeds[2]<=xi)*states[1,3]
            return h_out, hu_out
        
    elif transonic == 2:
        beta = (lambda_2_r - s2)/(lambda_2_r - lambda_2_m)
        #alpha2_l = beta*alpha2
        alpha2_r = (1-beta)*alpha2
        h_m_new = q_r[0] - alpha2_r
        hu_m_new = q_r[1] - alpha2_r*(u_hat+c_hat)
        q_m_new = np.array([h_m_new, hu_m_new])
        states = np.column_stack([q_l,q_m,q_m_new,q_r])
        speeds = [s1,lambda_2_, lambda_2_r]
        wave_types = ['contact']*3

        def reval(xi):
            h_out  = (xi<speeds[0])*states[0,0] + \
                     (speeds[0]<=xi)*(xi<speeds[1])*states[0,1] + \
                     (speeds[1]<=xi)*(xi<speeds[2])*states[0,2] + \
                     (speeds[2]<=xi)*states[0,3]
            hu_out = (xi<speeds[0])*states[1,0] + \
                     (speeds[0]<=xi)*(xi<speeds[1])*states[1,1] + \
                     (speeds[1]<=xi)*(xi<speeds[2])*states[1,2] + \
                     (speeds[2]<=xi)*states[1,3]
            return h_out, hu_out
        
    else:    
        states = np.column_stack([q_l,q_m,q_r])
        speeds = [s1, s2]
        wave_types = ['contact','contact']
    
        def reval(xi):
            h_out  = (xi<s1)*h_l + (s1<=xi)*(xi<=s2)*h_m + (s2<xi)*h_r
            hu_out = (xi<s1)*hu_l + (s1<=xi)*(xi<=s2)*hu_m + (s2<xi)*hu_r
            return h_out, hu_out
    
    return states, speeds, reval, wave_types

Here are some examples of problems with transonic rarefactions, showing the behavior of the solver.

In [ ]:
h_l = 1; h_r = 1
u_l = 0.5; u_r = 2

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_roe_with_efix(q_l, q_r)
no_efix_states, no_efix_speeds, no_efix_reval, no_efix_wave_types = shallow_water_roe(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states, no_efix_states],
                                                 [ex_speeds, speeds, no_efix_speeds],
                                                 [ex_reval, reval, no_efix_reval],
                                                 [ex_wave_types, wave_types, no_efix_wave_types],
                                                 #['Exact','Roe with fix','Roe w/o fix'],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=no_efix_states)

In [ ]:
h_l = 4; h_r = 1
u_l = 1; u_r = 1

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_roe_with_efix(q_l, q_r)
no_efix_states, no_efix_speeds, no_efix_reval, no_efix_wave_types = shallow_water_roe(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states, no_efix_states],
                                                 [ex_speeds, speeds, no_efix_speeds],
                                                 [ex_reval, reval, no_efix_reval],
                                                 [ex_wave_types, wave_types, no_efix_wave_types],
                                                 #['Exact','Roe with fix','Roe w/o fix'],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=no_efix_states)

### Positivity

In the shallow water system, it is both physically and mathematically essential that the depth remain non-negative.  Unfortunately, as the next example shows, the Roe solver does not always guarantee this.

In [ ]:
h_l = 1; h_r = 1
u_l = -1.5; u_r = 1.5

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_roe_with_efix(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states, hmin = -1)

In fact, this is a feature of all linearized Riemann solvers: for some Riemann problems, they will produce negative intermediate states.  The HLL solver described next does guarantee a positive solution for all initial data.

## HLL solver

### Derivation and implementation

Recall that an HLL approximate solver is based on a fictitious solution consisting of two waves (discontinuities) with a single intermediate state $q_m$ between them.  The waves are thus

\begin{align}
\wave_1 & = q_m - q_l \\
\wave_2 & = q_r - q_m.
\end{align}

The middle state is determined by the requirement of conservation:

\begin{align}
s_1 \wave_1 + s_2 \wave_2 & = f(q_r) - f(q_l),
\end{align}

which can be solved for $q_m$ to find

\begin{align}  \label{hll_middle_state}
q_m = \frac{f(q_r) - f(q_l) - s_2 q_r + s_1 q_l}{s_1 - s_2}.
\end{align}

The key remaining ingredient is the choice of the wave speeds $s_1, s_2$.  These should approximate the wave speeds appearing in the true Riemann solution.  In general, it is desirable for the purpose of stability that the speeds used be approximations of the fastest (left- and right-going) waves in the Riemann solution.  Indeed, in the original paper of Harten, Lax, and van Leer (cite) it was proposed to use lower and upper bounds on the true wave speeds.  However, overestimating the wave speeds leads to increased diffusion, and various refinements to the choice of wave speeds were subsequently proposed.

For the shallow water system, the 1-wave speed is always less than or equal to the 2-wave speed.  If the 1-wave is a rarefaction, then the minimum wave velocity is given simply by $\lambda_1(q_l)$, the characteristic speed at the left edge of the 1-rarefaction fan.  If the left-going wave speed is a shock, then minimum wave velocity is the shock speed.  We cannot determine the shock speed exactly without solving the Riemann problem -- just what we hope to avoid.  Instead, we can use an approximate shock speed based on the Roe average:

$$\hat{s}_1 = \hat{u} - \hat{c}.$$

Since we don't know whether the 1-wave should be a shock or a rarefaction, we use a lower bound on these two speeds, by taking their minimum:

$$s_1 = \min\left(\lambda_1(q_l), \hat{u}-\hat{c}\right).$$

Similarly, the second wave speed is given by

$$s_2 = \max\left(\lambda_2(q_r), \hat{u}+\hat{c}\right).$$

This completes the definition of the solver. Notice that if the states $q_r, q_l$ are connected by a single 2-shock, then the value of $s_2$ will be the exact shock speed (since the Roe speed is exact in this case, and the shock speed will be faster than the characteristic speed ahead of it).  Furthermore, by substituting the Rankine-Hugoniot conditions into \eqref{hll_middle_state}, one sees that the the middle state in this case will be $q_m = q_l$.  Thus the HLL solver gives the exact solution for this case.  Similarly, it yields the exact solution when the initial states are connected by just a 1-shock.  It shares this nice property with the Roe solver (as long as the wave speeds are chosen as just described).

In [ ]:
def shallow_water_hll(q_l, q_r, grav=1.):
    """
    HLL approximate solver for the shallow water equations.
    """
    h_l = q_l[0]
    hu_l = q_l[1]
    u_l = hu_l/h_l
    h_r = q_r[0]
    hu_r = q_r[1]
    u_r = hu_r/h_r
        
    # Roe averages
    h_hat = (h_r + h_l)/2.
    u_hat = (np.sqrt(h_r)*u_r + np.sqrt(h_l)*u_l) / (np.sqrt(h_r) + np.sqrt(h_l))
    c_hat = np.sqrt(grav*h_hat)

    lambda_1_l = u_l - np.sqrt(grav*h_l)
    lambda_2_r = u_r + np.sqrt(grav*h_r)
    
    s1 = min(lambda_1_l, u_hat - c_hat)
    s2 = max(lambda_2_r, u_hat + c_hat)
    
    h_m = (hu_r - hu_l - s2*h_r + s1*h_l)/(s1-s2)
    hu_m = (hu_r*u_r - hu_l*u_l + 0.5*grav*(h_r**2 - h_l**2) - s2*hu_r + s1*hu_l)/(s1-s2)
    q_m = np.array([h_m, hu_m])
    
    states = np.column_stack([q_l,q_m,q_r])
    speeds = [s1, s2]
    wave_types = ['contact','contact']
    
    def reval(xi):
        h_out  = (xi<s1)*h_l + (s1<=xi)*(xi<=s2)*h_m + (s2<xi)*h_r
        hu_out = (xi<s1)*hu_l + (s1<=xi)*(xi<=s2)*hu_m + (s2<xi)*hu_r
        return h_out, hu_out
    
    return states, speeds, reval, wave_types

### Examples

In [ ]:
# Dam-break example
h_l = 4; h_r = 1
u_l = 0; u_r = 0

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_hll(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

In [ ]:
# Isolated 2-shock example
h_l = 2.20698770767; h_r = 1.
u_l = 2.27057814896/h_l; hu_r = 0.

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_hll(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 #['Exact','Roe with fix','Roe w/o fix'],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

In [ ]:
# Transonic rarefaction example a
h_l = 1; h_r = 1
u_l = 0.5; u_r = 2

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_hll(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

Notice that, by design, the HLL wave always lies exactly on the leading edge of a rarefaction wave, when compared to the true solution.

### Entropy and positivity

It can be shown that the HLL solver always preserves positivity and does not require an entropy fix.  Both of these properties result in part from the fact that the HLL solver is more diffusive than the Roe solver.

In [ ]:
# Transonic rarefaction example b
h_l = 4; h_r = 1
u_l = 1; u_r = 1

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_hll(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)

In [ ]:
h_l = 1; h_r = 1
u_l = -1.5; u_r = 1.5

hu_l = h_l*u_l; hu_r = h_r*u_r
q_l = np.array([h_l, hu_l]); q_r = np.array([h_r, hu_r])
ex_states, ex_speeds, ex_reval, ex_wave_types = shallow_water.exact_riemann_solution(q_l,q_r)
states, speeds, reval, wave_types = shallow_water_hll(q_l, q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wave_types, wave_types],
                                                 variable_names=shallow_water.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

In [ ]:
shallow_water.phase_plane_plot(q_l,q_r,g=1.,y_axis='u',approx_states=states)